# Detectron2 Savanna Tree AI
Use this notebook to imort training data in COCO format, visualise labels, train a model using Detectron2 architecture and score test data to visualise performance. 


### Import dataset

In [ ]:
#Register metadata for annotations and images
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

from detectron2.data.datasets import register_coco_instances

In [ ]:
#Import tree dataset in COCO format. Give your dataset a name in the registered_data line 'insert name here'. 
#Specify in the register_coco_instances() line the location of the .json file for annotations and the folder with the images.
registered_data='<insert dataset name here>'

register_coco_instances(registered_data, {}, "C:/<insert file path here>/<insert file name>.json", "C:/<path to folder with training images>")

In [ ]:
dataset_metadata = MetadataCatalog.get(registered_data)
dataset_dicts = DatasetCatalog.get(registered_data)

In [ ]:
#Run this cell to check the data has been registered successfully and all the classes are included.
dataset_metadata

### Import and register evaluation dataset

In [ ]:
#Register evaluation dataset with unseen images for testing model.
registered_data_eval='data_eval'
register_coco_instances(registered_data_eval, {}, "C:/<insert file pathe here>/<insert file name>.json", "C:/<path to folder with training images>")

In [ ]:
#Register evaluation dataset metadata
dataset_metadata_eval = MetadataCatalog.get(registered_data_eval)
dataset_dicts_eval = DatasetCatalog.get(registered_data_eval)

In [ ]:
#Run this cell to check the data has been registered successfully and all the classes are included.
dataset_metadata_eval

### Visualise annotations and labels

In [ ]:
#Run this cell to visualise 2 random images from your training dataset with their annotations and labels overlayed.
#Check for any anomalies in your data before continuing. Re-run cell to view different labels. 
import random
import cv2
%matplotlib inline
import matplotlib.pyplot as plt

from detectron2.utils.visualizer import Visualizer



#for d in random.sample(dataset_dicts, 3):
for d in dataset_dicts_eval[2:4]:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata_eval,scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.figure(figsize = (20,20))
    plt.imshow(vis.get_image()[:,:, ::-1])
    plt.show()
    #plt.savefig('fig.jpg')
    print(d["file_name"])
    

# Train Detectron2 Model

In [ ]:
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer
class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = (registered_data,)
cfg.DATASETS.TEST = (registered_data_eval,)  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2


cfg.MODEL.WEIGHTS =model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl")
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 10000   #100000  # 200000   # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64  # faster, and good enough for this dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 38  #check the number of classes in your dataset when registering above
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 38 #Number of classes
cfg.TEST.EVAL_PERIOD = 100

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

#### View performance in tensorboard
To view plots of the metrics above e.g. loss etc. 

In a command promt type --> tensorboard --logdir=E:\es\Detectron2\Coding\SavannaTree\output Then copy and paste http://localhost:6006 into a web browser to view plots of model performance

# Make predictions from trained model

In [ ]:
#Run this cell to create a pedictor from the model trained above. 
#If making using this notebook to make predictions after closing kernel see below create predictor.

from detectron2.engine import DefaultPredictor


# Predicting (create predictor)
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2   # set the testing threshold for this model
cfg.DATASETS.TEST = (registered_data_eval, )
predictor = DefaultPredictor(cfg)


### Score random image from evaluation data

In [ ]:
#run this cell to view a pridction of a random image from the training data.
#Note: this will liekly show overfit data.
from detectron2.utils.visualizer import ColorMode
import random
import cv2
%matplotlib inline
import matplotlib.pyplot as plt

from detectron2.utils.visualizer import Visualizer

#for d in random.sample(dataset_dicts, 3): 
for d in dataset_dicts1[2:4]:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    vis = Visualizer(im[:, :, ::-1],
                   metadata=dataset_metadata_eval, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = vis.draw_instance_predictions(outputs["instances"].to("cpu"))
    #cv2_imshow(vis.get_image()[:, :, ::-1])
    plt.figure(figsize = (20,20))
    plt.imshow(v.get_image()[:,:, ::-1])
    plt.show()

### Score chosen image from desktop

In [ ]:
# Validate new image
#Use this cell to indicate the specific image you want to score. Will need to change the directory in "" below. 

im = cv2.imread("<path to file here e.g. C://User//test.png>")
outputs = predictor(im)
vis = Visualizer(im[:, :, ::-1],
               metadata=dataset_metadata1, 
               scale=0.8, 
               instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = vis.draw_instance_predictions(outputs["instances"].to("cpu"))
#cv2_imshow(vis.get_image()[:, :, ::-1])
plt.figure(figsize = (20,20))
plt.imshow(v.get_image()[:,:, ::-1])
plt.show()

# Make predictions from model (Post-training)

If you have trained a model and closed the notebook you need to recreate a predictor with all the dependencies below, without having to retrain the model again. 
The cells below are setup to make predictions on a random image in the evaluation dataset if registered or a specified image on your harddrive.  

In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
import os
import random
import cv2
%matplotlib inline
import matplotlib.pyplot as plt


# Create config
cfg = get_cfg()
cfg.merge_from_file("/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = (registered_data, )
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 38
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 38

# Create predictor
predictor = DefaultPredictor(cfg)

In [ ]:

from detectron2.utils.visualizer import ColorMode


#for d in random.sample(dataset_dicts, 3): 
for d in dataset_dicts[2:4]:
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    vis = Visualizer(im[:, :, ::-1],
                   metadata=dataset_metadata_eval, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = vis.draw_instance_predictions(outputs["instances"].to("cpu"))
    #cv2_imshow(vis.get_image()[:, :, ::-1])
    plt.figure(figsize = (20,20))
    plt.imshow(v.get_image()[:,:, ::-1])
    plt.show()

In [ ]:
# Validate new image
#im = cv2.imread(d["file_name"])

im = cv2.imread("<insert test file>")
outputs = predictor(im)
vis = Visualizer(im[:, :, ::-1],
               metadata=dataset_metadata_eval, 
               scale=0.8, 
               instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = vis.draw_instance_predictions(outputs["instances"].to("cpu"))
#cv2_imshow(vis.get_image()[:, :, ::-1])
plt.figure(figsize = (20,20))
plt.imshow(v.get_image()[:,:, ::-1])
plt.show()

# Score Video

In [ ]:
#Run this cell to score a video.

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import tqdm
import cv2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog
import time

video = cv2.VideoCapture('E://<path to video>/.mp4')
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))


video_writer = cv2.VideoWriter('scoredvideo.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)


cfg = get_cfg()
cfg.merge_from_file("/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = (registered_data, )
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 9
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 9
predictor = DefaultPredictor(cfg)


v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TEST[0]), ColorMode.IMAGE)

def runOnVideo(video, maxFrames):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """

    readFrames = 0
    while True:
        hasFrame, frame = video.read()
        if not hasFrame:
            break

    
        outputs = predictor(frame)

       
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

       
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

        readFrames += 1
        if readFrames > maxFrames:
            break


num_frames = 720


for visualization in tqdm.tqdm(runOnVideo(video, num_frames), total=num_frames):

   
    cv2.imwrite('scored_detectron2.png', visualization)

    
    video_writer.write(visualization)


video.release()
video_writer.release()
cv2.destroyAllWindows()

# Evaluate dataset (Evaluation Dataset)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator(registered_data_eval, cfg, False, output_dir="./output/COCO_evaluator")
val_loader = build_detection_test_loader(cfg, registered_data_eval)
inference_on_dataset(trainer.model, val_loader, evaluator)